In [ ]:
#autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))
learn = vision_learner(dls, resnet34, metrics=error_rate)
# learn.fine_tune(1, 3e-3)
# res = learn.validate()
# res 



In [ ]:
learn.cbs

In [ ]:
res[0]

In [ ]:
from data.unlabeled import OpenWebTextConfig, TiktokenTokenizer
from model.gpt2 import GPT
from train import OWTData, RandomSubsetSampler, save_model_checkpoints
import numpy as np
from fastai.text.all import *
from torch.utils.data import DataLoader

In [ ]:
from model.customLearner import customLearner

In [ ]:
tokenizer = TiktokenTokenizer()

In [ ]:

bs = 16
block_size = 512
valid_sampler_size = 1000 #how many samples to use for validation. This is only used to check if validation loss is better than best_valid_loss, so that a checkpoint can be saved. Karpathy uses 200 random points



model = GPT(block_size=block_size)

train_ds = OWTData(OpenWebTextConfig().default_cache_dir/'train.bin'    ,block_size=block_size, dtype = tokenizer._get_numpy_dtype())
valid_ds = OWTData(OpenWebTextConfig().default_cache_dir/'val.bin'      ,block_size=block_size, dtype = tokenizer._get_numpy_dtype())
                    
train_dl = DataLoader(train_ds, batch_size=bs)

valid_dl = DataLoader(valid_ds, batch_size=2*bs,
                        sampler = RandomSubsetSampler(valid_ds, subset_size=valid_sampler_size),
                        )


dls = DataLoaders(train_dl, valid_dl)
dls.c = model.head.out_features


check_and_save_model = save_model_checkpoints(dir = 'checkpoints', 
                                                model_name = str(model), 
                                                checkpoint_name = 'gpt2', 
                                                every_iters = 10000)

learn = customLearner(dls, 
                model, 
                loss_func=CrossEntropyLossFlat(), 
                metrics=[accuracy, Perplexity()],
                cbs = [check_and_save_model],
                ).to_bf16()

In [ ]:
learn.fit_one_cycle(1, 1e-4)

In [ ]:
learn.fit_one_cycle(1, 1e-4)

In [ ]:
x,y = next(iter(train_dl))
tokenizer.decode()

In [ ]:
list(x[0])